# Import librairies

In [1]:
import requests
import tweepy
import webbrowser
import time
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# Import keys

In [2]:
from raw_data import keys

In [3]:
twitter_api_key = keys.twitter_api_key()
twitter_api_secret_key = keys.twitter_api_key_secret()
twitter_bearer_token = keys.twitter_bearer_token

# Connect my developer project to my twitter profile

In [4]:
callback_uri = "oob" # url
auth = tweepy.OAuthHandler(twitter_api_key, twitter_api_secret_key, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=LEVZ_gAAAAABUe3MAAABfOpq7bM


In [5]:
m=

SyntaxError: invalid syntax (1928120398.py, line 1)

In [5]:
user_pin_value = '4046236'
auth.get_access_token(user_pin_value)
api = tweepy.API(auth)

# Load new content

## Function to extract the tweets of politicals that I follow

In [169]:
def extract_tweets_from_politics(number_of_tweet_per_politic):
    """This function return a DataFrame of the last tweets of politics that I follow on Twitter"""
    
    # Start by working on my own account
    me = api.get_user(screen_name="alecoursonnois")
    # Generate a list of my all of my friends (considering that I have under 50 friends on twitter)
    my_friends = me.friends(count=50)
    
    # Retrieve my personal friends which I don't want to analyze their tweets
    not_politics = ["alecoursonnois", "Baly_5", "LucieCMP", "JeannonoSmith", "Seezzy_"]
    
    # Create a set to stock the DataFrame columns names
    columns = set()
    # Create a list to save each tweet and its datas
    tweets_data = []
    # Create to save politics screen_names
    politics_screen_name = []
    
    # Iterate on my_friends
    for friend in my_friends:
        friend_name = friend.screen_name
        
        # Don't act if this friend is out of the project
        if friend_name in not_politics:
            pass
        
        else:
            # Generate politics timeline list of the last tweets (the number must be indicated as a function's argument)
            timeline_list = api.get_user(screen_name=friend_name).timeline(count=number_of_tweet_per_politic, tweet_mode="extended")
            
            # Iterate on each tweet
            for status in timeline_list:
                # Generate a dictionary of status attributes
                status_dict = dict(vars(status))
                
                # From this dictionary, get the keys and stock it in a variable
                keys = vars(status).keys()
                
                # Create a dictionary stocking each status of the politic we are working on
                # This dictionary will be added to the "tweets_data" list and finaly convert into a DataFrame
                single_tweet_data = {}
    
                # Iterate on each status key (future column name)
                for k in keys:
                    # Add the status data to its right key into the dictionary of each status
                    single_tweet_data[k] = status_dict[k]
                    # Add the key as a column name in the set
                    columns.add(k)
                
                # Append the full single_tweet_data dictionary with the right key to the tweets_data list
                tweets_data.append(single_tweet_data)

    # Give a name to the columns respecting the order
    headers_cols = list(columns)

    # Allow the DataFrame to display full tweets when printed
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    
    # Convert the tweets_data list into a DataFrame
    df = pd.DataFrame(tweets_data)
    
    # Only keep columns that provides information
    columns_to_keep = ['id', '_json', 'source', 'source_url', 'retweet_count']
    df = df[columns_to_keep]
    
    # Generate new features from extracted data
    df["user"] = df["_json"].apply(lambda x : x["user"]["screen_name"])
    df["created_date"] = pd.to_datetime(df["_json"].apply(lambda x : x["created_at"])).dt.strftime("%Y-%m-%d")
    df["created_hour"] = pd.to_datetime(df["_json"].apply(lambda x : x["created_at"])).dt.strftime("%H:%M:%S")
    df["text"] = df["_json"].apply(lambda x : x["full_text"])
    df["hashtags"] = df["_json"].apply(lambda x : [hasthstags["text"] for hasthstags in x["entities"]["hashtags"]])
    
    # Reorder columns
    right_order = ["user", "id", "source", "text", "created_date", "created_hour", "hashtags", "retweet_count"]
    df = df[right_order]
    
    return df

## Apply the function

In [170]:
# Load new tweets
last_tweets_df = extract_tweets_from_politics(number_of_tweet_per_politic=250)

## Add features

- Parti politique 
- Tendance politique
- Date d'extraction (pour intégrer cette information il faudrait créer une copie de l'archive sans cette colonne pour pouvoir retirer les doublons, puis, merger le df sans doublon au sauvegardé pour avoir la date d'extraction)
- Score par lexique (plus tard)

### Generate new_features

In [173]:
new_features = [{"user":'EPhilippe_LH', "name": "Édouard Philippe","political_party":"horizons", "political_trend":"right"},
                 {"user":'ZemmourEric', "name": "Eric Zemmour", "political_party":"zemmour", "political_trend":"extrem_right"},
                 {"user":'EmmanuelMacron', "name": "Emmanuel Macron", "political_party":"LREM", "political_trend":"right"},
                 {"user":'Waechter2022', "name": "Antoine Waechter", "political_party":"mouvement_ecologiste_independant", "political_trend":"ecology"},
                 {"user":'Fabien_Roussel', "name": "Fabien Roussel", "political_party":"PCF", "political_trend":"extrem_left"},
                 {"user":'PhilippePoutou', "name": "Philippe Poutou", "political_party":"nouveau_parti_anticapitaliste", "political_trend":"extrem_left"},
                 {"user":'jfpoisson78', "name": "Jean-Frédéric Poisson", "political_party":"la_voie_du_peuple", "political_trend":"extrem_right"},
                 {"user":'f_philippot', "name": "Florian Philippot", "political_party":"les_patriotes", "political_trend":"extrem_right"},
                 {"user":'vpecresse', "name": "Valérie Pécresse", "political_party":"soyons_libres", "political_trend":"right"},
                 {"user":'DenisPayre', "name": "Denis Payre", "political_party":"les_republicains", "political_trend":"right"},
                 {"user":'montebourg', "name": "Arnaud Montebourg", "political_party":"l_engagement", "political_trend":"left"},
                 {"user":'JLMelenchon', "name": "Jean-Luc Mélenchon", "political_party":"la_france_insoumise", "political_trend":"extrem_left"},
                 {"user":'MLP_officiel', "name": "Marine Le Pen", "political_party":"front_national", "political_trend":"extrem_right"},
                 {"user":'SLeFoll', "name": "Stéphane Le Foll", "political_party":"parti_socialiste", "political_trend":"left"},
                 {"user":'jeanlassalle', "name": "Jean Lassale", "political_party":"resistons", "political_trend":"divers"},
                 {"user":'larrouturou', "name": "Pierre Larrouturou", "political_party":"nouvelle_donne", "political_trend":"left"},
                 {"user":'philippejuvin', "name": "Philippe Juvin", "political_party":"les_republicains", "political_trend":"right"},
                 {"user":'yjadot', "name": "Yannick Jadot", "political_party":"EELV", "political_trend":"ecology"},
                 {"user":'Anne_Hidalgo', "name": "Anne Hidalgo", "political_party":"parti_socialiste", "political_trend":"left"},
                 {"user":'gerardfiloche', "name": "Gerard Filoche", "political_party":"la_gauche_democratique_et_sociale", "political_trend":"extrem_left"},
                 {"user":'dupontaignan', "name": "Nicolas Dupont-Aignan", "political_party":"debout_la_france", "political_trend":"extrem_right"},
                 {"user":'ECiotti', "name": "Eric Ciotti", "political_party":"les_republicains", "political_trend":"right"},
                 {"user":'xavierbertrand', "name": "Xavier Bertrand", "political_party":"la_manufacture", "political_trend":"right"},
                 {"user":'MichelBarnier', "name": "Michel Barnier", "political_party":"les_republicains", "political_trend":"right"},
                 {"user":'UPR_Asselineau', "name": "François Asselineau", "political_party":"union_populaire_republicaine", "political_trend":"divers"},
                 {"user":'n_arthaud', "name": "Nathalie Arthaud", "political_party":"lutte_ouvriere", "political_trend":"extrem_left"},
                 {"user":'MartinRocca12', "name": "Martin Rocca", "political_party":"constituante_2022", "political_trend":"divers"},
                 {"user":'gilleslazzarini', "name": "Gilles Lazzarini", "political_party":"parti_politique_pour_la_paix_et_la_protection_de_la_planete", "political_trend":"divers"},
                 {"user":'AlexLanglois_', "name": "Alex Langlois", "political_party":"refondation_2022", "political_trend":"divers"},
                 {"user":'luclaf', "name": "Luc Laforets", "political_party":"une_perspective_la_6e_republique", "political_trend":"divers"},
                 {"user":'Vukuzman', "name": "Georges Kuzmanovic", "political_party":"republique_souveraine", "political_trend":"divers"},
                 {"user":'FabriceGrimal', "name": "Fabrice Grimal", "political_party":"la_concorde_citoyenne_2022", "political_trend":"divers"},
                 {"user":'ClaraEgger1', "name": "Clara Egger", "political_party":"espoir_RIC_2022", "political_trend":"divers"},
                 {"user":'Cau_Marie_', "name": "Marie Cau", "political_party":"sans_etiquette", "political_trend":"divers"},
                 {"user":'antoine27955080', "name": "Antoine Martinez", "political_party":"volontaires_pour_la_france", "political_trend":"extrem_right"},
                 {"user":'regis_ollivier', "name": "Régis Ollivier", "political_party":"independant", "political_trend":"right"},
                 {"user":'JaclineMouraud', "name": "Jacline Mouraud", "political_party":"sans_etiquette", "political_trend":"right"},
                 {"user":'HeleneThouy', "name": "Hélène Thouy", "political_party":"parti_animaliste", "political_trend":"ecology"},
                 {"user":'MAZUEL_Pace', "name": "Philippe Mazuel", "political_party":"pace", "political_trend":"left"},
                 {"user":'MarCharlott', "name": "Charlotte Marchandise", "political_party":"sans_etiquette", "political_trend":"left"},
                 {"user":'AguebPorterie', "name": "Anna Agueb-Porterie", "political_party":"aucun", "political_trend":"left"},
                 {"user":'AnasseKazib', "name": "Anasse Kazib", "political_party":"courant_communiste_revolutionnaire_revolution_permanente", "political_trend":"extrem_left"}]

In [174]:
new_features_df = pd.DataFrame(new_features)

### Merge it to last_tweets_df

In [175]:
last_tweets_df = pd.merge(left=last_tweets_df, right=new_features_df, on='user', how='left')

## Check the number of tweets per politics

In [177]:
# last_tweets_df["user"].value_counts()

## Add it to the main global DataFrame

### Load the global_df 
- and drop the last index column automatically named "Unamed: 0" when saving df to csv file

In [188]:
global_df = last_tweets_df.copy()

In [129]:
global_df = pd.read_csv("global_df.csv").drop(columns="Unnamed: 0")

### Add new content

In [ ]:
# # Replace Nan values for retweet_count and favorite_count to 0
# global_df[["retweet_count"]] = global_df[["retweet_count"]].fillna("0")
# # Turn it to integers values
# global_df[["retweet_count"]] = global_df[["retweet_count"]].astype(int)

In [189]:
last_tweets_df.columns

Index(['user', 'id', 'source', 'text', 'created_date', 'created_hour',
       'hashtags', 'retweet_count', 'name', 'political_party',
       'political_trend'],
      dtype='object')

In [190]:
actual_number_of_tweets = global_df.shape[0]

# Append last tweets loaded to our archives
global_df = global_df.append(last_tweets_df, ignore_index=True)

# Sort the df by retweet_count to be sure to only keep the last update of retweet_count
global_df = global_df.sort_values('retweet_count', ascending=False).drop_duplicates(["user", "id", "source", "text", "created_date", "created_hour", "hashtags", "name", "political_party","political_trend"])

print(f"We just added {global_df.shape[0]-actual_number_of_tweets} tweets to our global_df")

TypeError: unhashable type: 'list'

### Save the new global_df

In [279]:
global_df.shape

(14486, 10)

In [280]:
global_df.to_csv("global_df.csv")

# Clean tweets

## Make a new df from the global one
- This way we will be allowed to drop duplicates next time we will load the global_df from the csv file

In [281]:
df_ready_for_nlp = global_df.copy()

## Retrieve recurrent regex

### Retweets

In [24]:
def retrieve_rt_retweets(df):
    new_texts = []
    
    for text in df["text"]:
        if text[:3] == 'RT ' or text[:3] == 'rt ':
            new_texts.append(text[3:])
        else:
            new_texts.append(text)
    
    df["text"] = new_texts
    
    return df

In [25]:
df_ready_for_nlp = retrieve_rt_retweets(df_ready_for_nlp)

### Links

In [26]:
def retrieve_links(df):
    pattern = re.compile("https://*")
    new_text = []

    for text in df["text"]:
        if re.search(pattern, text) is None:
            new_text.append(text)
        else:
            match_position = re.search(pattern, text).span()[0]
            new_text.append(text[:(match_position-1)])

    df["text"] = new_text
    
    return df

In [27]:
df_ready_for_nlp = retrieve_links(df_ready_for_nlp)


## Retrieve punctuation, lower text, tokenize, stopwords

### Define a text_cleaner function

In [28]:
def text_cleaner(text):
    # Remove Punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') 
    
    # Lower Case
    lowercased = text.lower() 
    
    # Remove accents
    unaccented_string = unidecode.unidecode(lowercased) 
    
    # Tokenize
    tokenized = word_tokenize(unaccented_string) 
    
    # Remove numbers
    words_only = [word for word in tokenized if word.isalpha()] 
    
    # Make stopword list
    stop_words = set(stopwords.words('french')) 
    
    # Remove Stop Words
    without_stopwords = [word for word in words_only if not word in stop_words]
    
    return " ".join(without_stopwords)

### Apply the text_cleaner function

In [29]:
df_ready_for_nlp['text'] = df_ready_for_nlp['text'].apply(text_cleaner)

# Analyze tweets

## Find lexical themes and store their vocabulary in lists

In [183]:
political_lexicon = """Zinoviev,Yougoslavie,sociales,yougoslave,volonté,vision,vietnamien,Vietnam,Varsovie,utopie,URSS,univers,union,UEC,Twitter,trotskyste,trotskysme,Trotsky,trotskiste,trotskisme,Trotski,troïka,tribalisme,totalitarisme,totalitaire,Tito,titiste,titisme,Thucydide,Thorez,termite,temporisation,tempête,technocratie,technocrate,Tchéka,Tchécoslovaquie,tchécoslovaque,Tchang,Taïwan,tabou,système,syndicalisme,sympathisant,stratification,stratégie,staliniste,stalinisme,stalinien,Staline,sphère,spartakiste,soviétomane,soviétique,soviet,souverainiste,souverainisme,souveraineté,Solidarnosc,solidarité,soin,sociologue,sociologique,sociologie,sociolecte,société,sociétal,socialiste,socialisme,socialisation,sociale,social,sociable,sociabilité,situation,sionisme,siège,Shanghai,SFIO,séparatiste,sentiment,sénat,secrétaire,sécession,scolaire,scission,schisme,scandale,samu,samizdat,ruse,RPC,Rousseau,Roumanie,rouge,révolutionnaire,révolution,restauration,responsabilité,respect,résistant,résistance,réseautage,république,républicain,religion,rejet,réinsertion,régime,réfugié,réformiste,réforme,réflexion,réactionnaire,réaction,RDA,rapport,rang,racial,querelles,qualité,publiciste,public,psychologique,psychique,province,protectorat,propagande,prolétarienne,progressiste,progressisme,progrès,programmes,programme,problèmes,pression,présidium,prélèvement,précarité,Pravda,pouvoir,postcommunisme,populiste,populisme,populaire,politologue,politologie,politiste,politiser,politisation,politiquement,politique,politicien,politicard,politicaillerie,politicailler,politburo,polémique,pluralisme,plan,philosophique,philosophie,philosophe,phénomènes,personnalités,perestroïka,Pékin,PCUS,PCI,pauvre,patriarcat,partisan,parti,partenaire,partageux,parlementarisme,parlementaire,parlement,parenté,paradigme,panslavisme,panafricanisme,pamphlet,pacte,orthodoxe,orientation,opposant,opportunisme,opinion,ONG,oligarchie,office,notion,nomenklatura,noblesse,nivellement,nihilisme,négociation,nationalité,nationaliste,nationalisme,nationalisation,nationale,nation,mutualité,municipale,multipartisme,moscoutaire,Moscou,moralité,morales,morale,moral,mondialisation,monarchiste,monarchie,mœurs,modéré,modèle,mixité,ministère,militant,milieu,méritocratie,menées,média,Mauss,matriarcat,marchais,maoïste,maoïsme,Mao,Malenkov,majorité,machiavélisme,machiavélique,Machiavel,maccarthysme,lutte,logement,locatif,littérature,liens,liberté,libertaire,libération,libéral,liant,léniniste,léninisme,Lénine,légitimité,légitimation,leader,LCR,langue,laïcité,Kuomintang,KPD,Komintern,Kominform,Khrouchtchev,Kampuchéa,justice,juridique,Juppé,junte,journal,Jiangxi,jeunesse,isolationnisme,islamisme,intrigues,interventionnisme,interpellation,internationaliste,internationale,intérieure,intérêts,interaction,intellectuel,intégration,institutions,institutionnel,institution,instabilité,insertion,infléchir,inégalité,indochinois,individuel,individualiste,individu,indépendantiste,indépendantisme,indépendance,impérialiste,impérialisme,immigration,idéologique,idéologie,identité,identitaire,idéal,humain,Hoxha,Hobbes,HLM,histoire,hiérarchie,handicap,habitus,habitude,habitat,habitant,habileté,Guomindang,guérilla,Guépéou,groupes,grégaire,gouverner,gouvernementale,gouvernement,gouvernant,Gottwald,Gorbatchev,glasnost,géopolitique,gazette,gaulliste,gaullisme,gauchiste,gauche,FTP,fourmi,fondement,fiscal,féodalité,féminisme,fédération,fédéralisme,fédérale,fascisme,familial,faction,facteurs,extérieure,européenne,étranger,ethnologie,ethnique,éthique,état,établissement,environnemental,entrisme,entente,Engels,endogamie,encarté,émergence,élection,électeur,égalité,égalitaire,effondrement,éducatif,éducateur,éditorial,échiquier,Durkheim,dumping,Duclos,droit,Doriot,domaine,dissident,dissidence,discrimination,dirigeant,diplomatique,diplomatie,diplomate,dictature,dialogue,déviationniste,déviance,développement,déterminisme,déstalinisation,dépolitiser,démographique,démocratique,démocratie,démocrate,démagogie,défavorisé,déclassement,décentralisation,débat,culturel,Cuba,critique,courant,corporatisme,coopération,coopératif,convictions,conversion,contexte,contestation,contestataire,constructivisme,constitutionnelle,constitution,considérer,considérations,conservatisme,conservateur,consensus,congrès,conformisme,conflits,conflit,conduite,concept,comportemental,comportement,compétence,communiste,communisme,communautaire,commerciales,comité,colonial,collectivité,collectiviste,collective,cohésion,cognitif,codes,coco,coalition,clivage,clientélisme,cléricalisme,classe,clandestinité,clandestin,clan,civique,civilisation,civil,citoyen,chinois,Chine,changement,CGTU,centriste,CCAS,Castro,castriste,caste,cassos,capitaliste,capitalisme,camarade,Cabet,bureaucratie,Brejnev,bourgeoisie,bourgeoise,bourgeois,Bourdieu,bouleversement,Boudon,bonapartiste,Bonald,bolcheviste,bolchevique,bolchevik,Blum,bipolarisation,bienfaisance,Berlinguer,bénéficiaires,bailleur,babouviste,avantage,autruche,autoritaire,autonomiste,autonomie,autonome,autocratie,autarcie,Aubervilliers,athées,associatif,assemblées,artistiques,argot,arène,architecture,appartenance,apolitique,apartheid,antisocial,antiparlementaire,antifasciste,anticommuniste,anticommunisme,anticléricalisme,anthropologue,anthropologique,anthropologie,anomie,anarchiste,anarchisme,anarchie,analyse,amnistie,ambitions,ambition,ambitieuse,ambiant,alpha,alliance,aliénation,Albanie,aimable,agrarien,agraire,agora,agitation,affable,administrative,adhérent,actualité,activiste,activisme,accession,abus,abstentionniste"""
political_lexicon = word_tokenize(text_cleaner(political_lexicon))

In [184]:
ecology_lexicon = """altermondialiste,agroalimentaire,agriculture,vivant,vert,végétale,trophique,surpopulation,recyclage,productivisme,préservation,pollution,phytogéographie,permaculture,organisme,oikos,océanographie,naturel,nature,microbiologie,mésologie,Mei,intertidale,impact,Hulot,guilde,Greenpeace,génération,évolution,éthologie,ethnographie,environnementale,environnement,énergie,écosystème,éconavigation,écologue,écologiste,écologisme,écologique,écologie,écoguerrier,écocitoyenneté,durable,diversité,décroissance,cours,bobo,biotope,biotique,biosphère,bionomie,biomasse,biologique,biologie,biogéographie,biodiversité,biocénose,agricole,abiotique"""
ecology_lexicon = word_tokenize(text_cleaner(ecology_lexicon))

In [185]:
economic_lexicon = """Walras,vitale,Turgot,Trente,touristique,tourisme,tirelire,thésaurisation,théoricien,textile,tertiaire,tempérance,tabac,surproduction,surplus,substantielles,structurel,stimuler,sobriété,Smith,secteur,sciences,Schumpeter,rural,Ricardo,ressources,réserve,relever,relancer,relance,redresser,redressement,reconversion,récession,réaliser,Quesnay,québécoise,publiques,protectionnisme,prospère,prévoyance,planification,physiocrate,pétrolier,pétrole,parcimonie,œconomie,OCDE,Nobel,NEP,néoclassique,naturelles,mondiale,modernisation,Milton,microéconomique,microéconomie,mesure,mercantiliste,mercantilisme,ménage,mathématique,matelas,marchande,Malthus,magot,macroéconomique,macroéconomie,libéraliser,libéralisation,libérale,keynésien,Keynes,interdépendance,informel,industrielle,industrialisation,importation,importance,historicisme,hétérodoxe,Hayek,harmonie,Glorieuses,globalisation,gisement,gestion,gaspillage,frugalité,Friedman,FMI,florissante,finances,feuille,faillite,externalité,exsangue,exportation,exportateur,étatique,essor,épargner,enjeu,endogène,élevage,efficience,économisme,économiseur,économiser,économiques,économiquement,économique,economics,économétrie,économe,école,échelle,dynamisme,dynamique,dollar,diversification,distribution,disposition,dirigisme,dépenser,démographie,déficit,cybernétique,côté,corruption,coordination,constat,consommateur,conjoncture,concision,compression,compétitivité,commercial,commerce,collectivisation,chandelle,canadienne,cagnotte,budget,boursicot,Boukharine,boom,Bastiat,avarice,autrichienne,autosuffisance,assurance,arrangement,approche,anticipation,alimentaire,administration,Adam,Vivendi,versement,trésorerie,trésor,transition,totalité,tendance,technologie,taxation,taux,surcapitalisation,subsistance,stratégique,stock,statistique,spéculation,souscripteur,solidaire,service,SARL,rôle,richesse,ressource,répartition,rentier,rente,rentabilité,rendement,rembourser,remboursement,réduction,redistribution,reconstruction,recette,rachat,protectionniste,propriété,propriétaire,profit,production,productif,privés,privatisation,primordial,prêteur,possessions,possédant,PME,placement,PIB,pécule,pêchés,payable,patrimoine,partners,paiement,organisation,numéraire,multinationale,montant,monopole,monnaie,monétaire,milliard,marxiste,marxisme,Marx,marchandise,majoritaire,maître,luxure,liquidité,libéralisme,krach,keynésianisme,investisseur,investissement,investir,intérêt,infrastructure,inflation,industriel,industrie,impôt,imposable,importants,immobiliser,immobilisation,immobilier,immatériel,holding,gains,fructifier,fortune,fonds,foncier,fiscalité,financier,financement,finance,filiale,excédent,euro,Eurazeo,espèces,escompte,épargne,épargnant,entreprise,entreprenant,engagés,endettement,emprunteur,emprunt,économiste,économie,échéance,échange,dollars,dividende,dissoudre,disponible,diminution,dévaluation,dette,détenu,détenteur,détenir,déflation,décupler,décisif,croissance,crise,crédit,créancier,coût,contribuable,concurrence,comptabilité,commanditer,circulation,chef,cession,central,cardinal,caractéristique,capitalistique,capitaliser,capitalisation,capital,caisse,budgétaire,boursier,bourse,bénéfice,banque,bancaire,autofinancement,austérité,augmentation,argent,appropriation,apporteur,apport,annuité,amortissement,amorçage,amenant,amalgames,allocation,affairer,actionnariat,actionnaire,acquisition,achat,accumuler,accumulation,accroissement"""
economic_lexicon = word_tokenize(text_cleaner(economic_lexicon))

In [186]:
extrem_lexicon = """xénophobie,violent,violence,vif,vieillesse,vertigineux,ultime,toundra,tétracorde,terrible,terreur,terminal,tension,température,susceptibilité,survie,surexcitation,surabondant,suprême,superlatif,soutenir,sommet,skinhead,simplicité,Sibérie,sévérité,sécheresse,risqué,rigueur,relooking,redoutable,rareté,raffinement,radical,raciste,racisme,puissance,prudence,prostration,profond,précipitation,préciosité,plafond,piètre,pénurie,passionné,particulier,paroxysmique,paroxysme,paresseux,pâleur,outrée,outrancier,opposition,opposé,onction,obligeance,nudité,nervosité,néonazi,nébulosité,musoir,mouvance,mortel,monstrueux,mollesse,miséricorde,minimal,minceur,metal,méprisable,méfiance,maximum,maximal,maigreur,lourdeur,Lolita,limite,lenteur,laideur,Laconie,Kolyma,intensif,intense,insupportable,insensibilité,inouï,injuste,inique,infini,individualisme,immodéré,immense,hypersexualisation,hypersensibilité,horrible,horreur,Heyman,héroïque,haut,hasardeux,groupuscule,gravité,glacial,furieux,funeste,fragilité,fou,fondamental,folle,finesse,férocité,fatigue,fasciste,faiblesse,facho,extrêmophile,extrémité,extrémiste,extrémisme,extrémiser,extrémisation,extrêmement,extrême,extraordinaire,extase,exquis,exorbitant,excessif,excès,exceptionnel,exaspéré,exagérer,épuisement,épouvantable,éperdu,énergique,électorat,effroyable,effréné,effrayant,ECW,dureté,drastique,draconien,disproportionné,disproportion,discrétion,détresse,désordonné,désolation,désespérer,désertique,dérégler,démesuré,délire,délicatesse,définitif,décrépitude,débilité,Couvin,contraire,comble,circonspection,Cabinda,Bonifacio,blanc,Bièvres,beauf,atrocité,asservir,aride,ardeur,arctique,anxiété,antisémitisme,antisémite,antipode,angoisse,aigu,agressivité,affreux,adoration,acharner,abusif,abstention,absolu,abattement"""
extrem_lexicon = word_tokenize(text_cleaner(extrem_lexicon))

In [187]:
health_lexicon = """science,santé,hygiène, vitamine, vitalité, VIH, vigueur, vieillissement, vie, vétérinaire, variole, valide, valétudinaire, vaccination, vaccin, urgence, UNICEF, tuberculose, trinquer, trempe, transfusion, traitement, toxique, thérapie, thérapeutique, tempérance, tempérament, tabagisme, tabac, syndrome, surpoids, stress, souffrante, souffrance, soin, soigner, soignant, sida, shiatsu, sensualité, sanitaire, sanatorium, samu, salutaire, salubrité, salubre, sain, robuste, risque, rétablissement, rétablir, reproductive, remède, régime, réadaptation, pulmonaire, psychologue, psychique, psychiatrie, protection, prospérité, prosit, probiotique, priorité, prévoyance, prévention, prescription, préoccuper, préoccupant, préjudiciable, praticien, pouls, potable, pollution, pneumonie, physique, pharmacien, pharmacie, pesticide, personnalité, patient, pathologie, paramédical, paludisme, organisme, organisation, OMS, obésité, nutritionnel, nutrition, nutriment, nuisible, nuisent, nuisance, nuigrav, nocif, naturopathie, naturopathe, naturel, nature, mutuelle, mutualité, mortalité, montagne, mondiale, métabolique, mental, ménager, médicament, médical, médecine, médecin, maternité, massage, malnutrition, malbouffe, malaria, malaise, maladif, maladie, malade, maigreur, longévité, jouir, invalidité, intoxication, interjection, institut, Inserm, insalubre, innocuité, inhalothérapeute, informatisation, infirmité, infirmière, infirmier, infectieux, infantile, indisposition, IMC, hypocondriaque, hypertension, hygiénisme, Hygie, humain, hospitalisation, hospitalière, hôpital, holistique, Hippocrate, health, handicap, guérison, guérir, gravement, généraliste, Galien, fumeur, fragile, force, florissant, évaluer, esprit, espérance, équilibre, épidémiologie, épidémie, environnementale, environnement, entretien, embonpoint, écosystème, domaine, disposition, dispensaire, diététique, diète, diagnostic, diabète, détériorer, détérioration, détenu, dépistage, dentisterie, dentiste, dentaire, délétère, décliner, curatif, covid, corporel, convalescence, contamination, consultation, constitution, consommateur, complémentaire, compétence, communautaire, clinique, chiropratique, cardiovasculaire, cardiaque, cancer, cadmium, bulletin, boire, biotechnologie, bile, asthme, assurance, Asclépios, aromathérapie, antioxydant, altérer, altération, allergie, allaitement, alimentation, alcoolisme, aggravé, absentéisme"""
health_lexicon = word_tokenize(text_cleaner(health_lexicon))

In [188]:
work_lexicon = """zèle,vacance,utilité,utilisation,usine,usage,turbin,travailleur,travailler,travail,transformation,Titan,thermodynamique,temps,télétravail,technique,taylorisme,taf,tâche,syndicat,syndicaliste,syndical,surmenage,surcroît,surcharge,supplémentaire,superviser,succès,stress,stakhanovisme,spécialité,smic,sinécure,semaine,séance,savoir,sape,salarié,salariat,revenu,retraite,résultat,repos,réparation,rémunéré,rémunération,relation,relâchement,récréation,reconnaissance,récompensé,recherche,réaménagement,quotidien,prud'hommes,prolétariat,prolétaire,projet,professionnel,profession,produit,productivité,prestation,présentéisme,préparatoire,pratique,planque,perfectionnisme,pensum,pénible,peine,paye,patronat,patron,paie,ouvroir,ouvrière,ouvrier,ouvrage,ouverture,outil,opération,oisiveté,œuvre,occuper,occupation,obligatoire,motivation,minutieux,métier,méthodologie,méthodique,méthode,mécanisation,mécanique,manufacture,manuel,management,machinisme,machine,loisir,licenciement,licencié,laborieux,laboratoire,labeur,journalier,joule,job,invalidité,intérim,inspection,inlassable,ingrat,indemnité,incessant,inachevé,horaire,heure,Hercule,Héraclès,grève,goulag,galère,gain,fruit,fordisme,forcé,forçat,fonctionnement,fonctionnaire,fonction,flexibilité,fastidieux,fainéantise,façonnage,exploitation,expérience,exigeant,exercice,exécution,étudier,étude,esclave,ergothérapie,ergonomie,éreintant,équipe,entretien,entrepreneur,entreprendre,enfantement,employeur,employer,employé,emploi,embellissement,embauche,élaboration,effort,effectuer,éducation,durée,donner,domestique,directeur,difficulté,devoir,devis,délassement,débordé,débaucher,création,corvée,convention,contrat,construction,consciencieux,congé,conciliation,conception,concentration,commission,colossal,collègue,collectivisme,collectif,collaboratif,collaborateur,chômeur,chômer,chômage,cheminement,charge,chantier,cabinet,bureau,brocante,bricole,bricolage,boulot,boîte,blouse,besogne,bénévole,bénévolat,bénédictin,bagne,bâcler,avancement,automatisation,atelier,assiduité,assidu,artisanat,artisan,ardu,apprentissage,appareil,amateur,affaire,activité,achèvement,acharné,accouchement,accomplir,absentéisme,abrutissant,aboutissement"""
work_lexicon = word_tokenize(text_cleaner(work_lexicon))

In [189]:
security_lexicon = """vulnérabilité,vigile,vigilance,vidéosurveillance,veto,vérification,veiller,usagers,urgence,unies,tranquillité,traçabilité,terroriste,terrorisme,Symantec,surveiller,surveillance,sûreté,stasi,stadier,stabilité,soupape,signalisation,sérénité,security,sécurité,securitas,sécuritarisme,sécuritaire,sécurisé,sécurisation,sécu,secourisme,Schengen,sauvetage,sauvegarde,sanitaire,salubrité,safety,routière,risquer,risque,ressortissant,responsable,résolution,répression,renseignement,renforcer,renforcement,réglementation,règlement,refuge,recommandation,rassurant,quiétude,protocole,protéger,protection,prospérité,procédure,problème,priorité,prévention,prévenir,présidence,police,piratage,piétons,pharmacovigilance,permanent,pénal,passive,parade,parachute,palestinien,paix,OTAN,ordre,OpenBSD,ONU,obligation,nucléaire,norme,NKVD,NFC,nécessaire,mousqueton,minutie,mesures,menacer,menace,maximale,maritime,manutention,manquement,malveillant,maintien,maintenance,loi,législation,KGB,ISO,Irak,investigation,intervention,international,intégrité,installation,insécurité,informatique,informaticien,incendie,impunité,humanitaire,harnais,hacker,gendarmerie,gendarme,garantie,FSB,freinage,fiabilité,FBI,faille,exigence,évaluation,évacuation,espionnage,escalade,ERP,englober,encadrement,domotique,divulgation,dispositif,détection,désarmement,déploiement,denrée,déminage,délinquance,défense,dangereux,danger,cybersécurité,cybercriminalité,cryptographie,criminalité,cracker,coussin,contrôles,continuum,conseil,conformité,confidentialité,confiance,conducteur,compromettre,commissariat,clignotant,CIA,chiffrement,chicane,chargé,certitude,certification,certificat,ceinture,biométrique,Beria,baudrier,balisage,autorité,authentification,attestation,attentat,ataraxie,assureur,assurer,assistance,approvisionnement,antivol,amélioration,alarme,airbag,airain,agression,agents,agence,aérien,accident,abri"""
security_lexicon = word_tokenize(text_cleaner(security_lexicon))

In [190]:
purchasing_power_lexicon = """TVA,taxes,tarifs,suppression,salariés,salaire,revenus,retraites,retraité,redonner,prix,pensions,pauvreté,mutuelles,modestes,ménages,loyers,loyer,inégalités,importés,fiscale,exonération,érosion,dépenses,dépense,cotisation,consommation,carburant,Berne,appauvrissement,actifs,acquérir"""
purchasing_power_lexicon = word_tokenize(text_cleaner(purchasing_power_lexicon))

In [191]:
other_candidates = ["ZemmourEric","yjadot","xavierbertrand","Waechter2022","Vukuzman","vpecresse","UPR_Asselineau","SLeFoll","regis_ollivier","PhilippePoutou","philippejuvin","n_arthaud","montebourg","MLP_officiel","MichelBarnier","MAZUEL_Pace","MartinRocca12","MarCharlott","luclaf","larrouturou","JLMelenchon","jfpoisson78","jeanlassalle","JaclineMouraud","HeleneThouy","gilleslazzarini","gerardfiloche","FabriceGrimal","Fabien_Roussel","f_philippot","EPhilippe_LH","EmmanuelMacron","ECiotti","dupontaignan","DenisPayre","ClaraEgger1","Cau_Marie_","antoine27955080","Anne_Hidalgo","AnasseKazib","AlexLanglois_","AguebPorterie"]

## Define a function to create a df with a lexicon score per user

### Store all the lexicons in a dictionary

In [192]:
lexicon_themes = {"political": political_lexicon, "ecology": ecology_lexicon, "economic":economic_lexicon, "extrem":extrem_lexicon, "health":health_lexicon, "work":work_lexicon, "security":security_lexicon, "purchasing_power": purchasing_power_lexicon, "other_candidates": other_candidates}

### Define the function

In [193]:
def lexicon_score_per_user(df, lexicon_themes):
    """Return a DataFrame scoring each tweet contents above a list of lexicon themes"""
    # Store the users in a list
    list_of_users = list(df["user"].unique())

    # Create a list to store each future row of the DataFrame
    user_scores = []

    # For each user
    for user in list_of_users:
        # Define a dictionary to store the score per lexicon per user
        lexicon_scores = {"user" : user}
        
        # Generate a score at zero for each theme
        for theme, lexicon in lexicon_themes.items():
            lexicon_scores[theme] = 0           
        
        # Extract his/her tweets
        user_tweets = list(df[df["user"]==user]["text"])
        # Then tokenize each tweet to analyze its content
        for tweet in user_tweets:
            if theme == other_candidates:
                tokenized_tweet = tweet
            else:
                tokenized_tweet = word_tokenize(tweet)
        
            # For each word
            for word in tokenized_tweet:
                # Is it part of one of the lexicons?
                for theme, lexicon in lexicon_themes.items():
                    if word in lexicon:
                        lexicon_scores[theme] += 1
        
        # Finally, append the user and its score per theme to user_scores
        user_scores.append(lexicon_scores)

    # Convert list to DataFrame
    lexicon_score_per_user_df = pd.DataFrame(user_scores)
    # Add political trend
    lexicon_score_per_user_df = pd.merge(left = lexicon_score_per_user_df, 
                                         right = df[["user", "name", "political_trend"]], 
                                         on = 'user', 
                                         how = 'left')
    # Drop duplicates due to merge just before
    lexicon_score_per_user_df = lexicon_score_per_user_df.drop_duplicates().reset_index(drop=True)
    
    return lexicon_score_per_user_df

### Apply it to df_ready_for_nlp

In [194]:
lexicon_score_per_user_df = lexicon_score_per_user(df=df_ready_for_nlp, lexicon_themes=lexicon_themes)
# lexicon_score_per_user_df

## Visualize the results

- For each user:
- One bar graph with his/her scores per lexicon
- One bar graph with his/her political trend average score per lexicon
- One bar graph with the global average score per lexicon

### Define the function

In [195]:
def visualize_scores(df, user):
    # Generate user score per theme
    user_scores = list(df[df["user"]==user].drop(columns="political_trend").iloc[0])[1:]
    
    # Generate user political trend average score per theme
    user_political_trend = list(df[df["user"]==user]["political_trend"])[0]
    # Generate his/her name
    user_name = list(df[df["user"]==user]["name"])[0]
    # Generate a df with the avg score per political trend
    avg_score_per_political_trend = df.groupby(['political_trend']).mean()
    # Store the right political trend results in a variable
    political_trend_scores = list(avg_score_per_political_trend
                                  [avg_score_per_political_trend.index==user_political_trend]
                                  .iloc[0])
    
    # Generate national score per theme
    national_scores = list(avg_score_per_political_trend.mean())
    
    # Define themes
    themes = avg_score_per_political_trend.columns
    
    fig = go.Figure(data=[
                    go.Bar(name=f'{user_name} score', x=themes, y=user_scores),
                    go.Bar(name=f'{user_political_trend} score', x=themes, y=political_trend_scores),
                    go.Bar(name='National score', x=themes, y=national_scores)
                    ])
    
    # Change the bar mode
    fig.update_layout(barmode='group')
    
    fig.show()

### Apply it

In [196]:
for political_user in list(df_ready_for_nlp["user"].unique()):
    visualize_scores(df=lexicon_score_per_user_df, user=political_user)

# Générer une map des candidats en fonction des champs lexicaux

In [197]:
# Select your 2 most important lexicons
first_lexicon = "ecology"
second_lexicon = "purchasing_power"

In [198]:
fig = px.scatter(lexicon_score_per_user_df, x=first_lexicon, y=second_lexicon, color="political_trend", hover_data=["name"])
fig.show()

# Nombre de tweets par utilisateur et en moyenne par candidat depuis septembre

In [199]:
tweets_per_user_df_name = pd.DataFrame(df_ready_for_nlp["name"].value_counts()).reset_index().rename({"name":"tweet_per_user", "index":"name"}, axis = 1)
tweets_per_user_df_name["average_number_of_tweets_per_candidate"] = int(sum(df_ready_for_nlp["name"].value_counts()/len(tweets_per_user_df_name["name"])))
print(sum(df_ready_for_nlp["name"].value_counts()))
tweets_per_user_df_name.head()

14176


,name,tweet_per_user,average_number_of_tweets_per_candidate
0,Gerard Filoche,1441,337
1,Anne Hidalgo,767,337
2,Jean-Luc Mélenchon,593,337
3,Anasse Kazib,523,337
4,Denis Payre,484,337


# Top 5 des mots par champs lexicaux par utilisateur

In [200]:
def top_five_words_per_theme_per_user(user):
    # Generate a list of all users
    list_of_users = list(df_ready_for_nlp["user"].unique())
    
    # Store all the user words in a list
    user_words = []
    # Extract all his/her tweets
    user_tweets = list(df_ready_for_nlp[df_ready_for_nlp["user"]==user]["text"])
    # Go for each tweet
    for tweet in user_tweets:
        for user in list_of_users:
            if user in tweet:
                user_words.append(user)
        # Tokenize each tweet to analyze its content
        tokenized_tweet = word_tokenize(tweet)
        # Go for each word
        for word in tokenized_tweet:
            # Append it to the list
            user_words.append(word)
    
    # For each vocabulary stored in lexicons check how many times it appears in user_words    
    # Define a list to store the top 5 words per lexicon
    top_five_words = []
    for theme, lexicon in lexicon_themes.items():
        test_dico ={}
        # For each word in the theme's lexicon
        for word in lexicon:
            # Count how many times it appears in user_words
            score = user_words.count(word)
            test_dico[word] = score
        sorted_list = sorted(test_dico.items(), key = lambda kv: kv[1], reverse=True)
        top_five_words.append(sorted_list)
        print(f"{theme} : {sorted_list[:5]}")

In [202]:
top_five_words_per_theme_per_user(user="larrouturou")

political : [('gauche', 12), ('justice', 11), ('sociale', 8), ('populaire', 7), ('changement', 7)]
ecology : [('vivant', 2), ('recyclage', 1), ('energie', 1), ('biodiversite', 1), ('altermondialiste', 0)]
economic : [('banque', 4), ('mondiale', 3), ('taux', 3), ('vitale', 2), ('industrielle', 2)]
extrem : [('radical', 2), ('immense', 2), ('effrayant', 2), ('abstention', 2), ('violence', 1)]
health : [('covid', 4), ('mondiale', 3), ('priorite', 2), ('vie', 1), ('protection', 1)]
work : [('semaine', 10), ('temps', 6), ('travail', 4), ('equipe', 4), ('donner', 4)]
security : [('loi', 4), ('priorite', 2), ('paix', 2), ('conseil', 2), ('verification', 1)]
purchasing_power : [('salaries', 5), ('retraites', 4), ('pauvrete', 4), ('prix', 2), ('loyer', 2)]
other_candidates : [('larrouturou', 68), ('yjadot', 6), ('xavierbertrand', 4), ('montebourg', 4), ('ZemmourEric', 0)]


# Top 3 des tweets par candidat en fonction du nombre de retweets

In [284]:
def top_3_tweets_per_user(df, user):
    df = df.sort_values(by="retweet_count", ascending=False)
    df = df["text"][df["user"]==user]
    print("Premier tweet le plus retweeter:")
    print(df.iloc[0])
    print("-------------")
    print("Second tweet le plus retweeter:")
    print(df.iloc[1])
    print("-------------")
    print("Troisième tweet le plus retweeter:")
    print(df.iloc[2])

In [285]:
top_3_tweets = top_3_tweets_per_user(df=global_df, user="larrouturou")
top_3_tweets

Premier tweet le plus retweeter:
140
-------------
Second tweet le plus retweeter:
140
-------------
Troisième tweet le plus retweeter:
140


In [286]:
text_test = "Prendre une balle de chasse dans la gorge au volant de sa voiture et se retrouver entre la vie et la mort, ce n'est pas la faute des #chasseurs. C'est la faute à pas de chance."

In [287]:
len(text_test)

176

# PCA pour associer des candidats en fonction de leurs scores

# + Carte d'identité de l'utilisateur avec :
- Sa photo
- Son nom
- Son parti politique
- Son nombre de tweets en archives depuis septembre (Vs le nombre moyen global par utilisateur)
- Son score par theme mis en perspective VS sa tendance politique et le natio
- Son top 5 des mots par champs lexicaux
- Son top 3 des tweets (en fonction du nombre de retweets)
- Son top 3 des tweets par thème (d'après une liste à sélectionner)

# + Affiner les lexiques, voir en ajouter